In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [15]:
# Step 1: Prepare the Mock Dataset (same as before)
years = 41
start_year = 1983
end_year = 2023

data = {
    'Year': list(range(start_year, end_year + 1)) * 15,
    'Country_A': ['USA', 'USA', 'USA', 'USA', 'USA', 'UK', 'UK', 'UK', 'UK', 'Canada', 'Canada', 'Canada', 'Italy', 'Italy', 'France'] * years,
    'Country_B': ['UK', 'Canada', 'Italy', 'France', 'Germany', 'Canada', 'Italy', 'France', 'Germany', 'Italy', 'France', 'Germany', 'France', 'Germany', 'Germany'] * years,
                  
    'Free_Trade': [1 for _ in range(15 * years)]
}

df = pd.DataFrame(data)

countries = list(set(df['Country_A'].tolist() + df['Country_B'].tolist()))
country_to_index = {country: i for i, country in enumerate(countries)}

edge_index = torch.tensor([[country_to_index[row['Country_A']], country_to_index[row['Country_B']]] for _, row in df.iterrows()], dtype=torch.long).t().contiguous()

edge_attr = torch.tensor(df['Free_Trade'].values, dtype=torch.float)

features = {
    'USA': [1.00, 21000, 2.0],
    'UK': [6.50, 14000, 3.0],
    'Germany': [0.85, 3800, 1.5],
    'Canada': [1.25, 1700, 2.5],
    'Italy': [110.00, 5000, 1.0],
    'France': [110.00, 5000, 1.0]
    
}

node_features = torch.tensor([features[country] for country in countries], dtype=torch.float)

exchange_rates = {
    'USA': 1.00,
    'UK': 6.50,
    'Germany': 0.85,
    'Canada': 1.25,
    'Italy': 110.00,
    'France': 110.00
}

previous_exchange_rates = {
    'USA': 1.00,
    'UK': 6.48,
    'Germany': 0.87,
    'Canada': 1.23,
    'Italy': 111.00,
    'France': 110.00
}

labels = torch.tensor([exchange_rates[country] - previous_exchange_rates[country] for country in countries], dtype=torch.float)


In [16]:
# Step 2: Define the Enhanced KA-GCN Model

class SuperKAGCN(nn.Module):
    def __init__(self, num_node_features, hidden_dim, output_dim, dropout=0.3):
        super(SuperKAGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.bn1 = BatchNorm(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = BatchNorm(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = BatchNorm(hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.fc2(x)
        return x


In [17]:
# Step 3: Implement Training with Advanced Techniques

def train_model(model, data, labels, num_epochs=500, patience=20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=10, min_lr=1e-6, verbose=True)
    criterion = nn.MSELoss()

    best_loss = float('inf')
    best_model = None
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
    
    model.load_state_dict(best_model)
    return model


In [18]:
# Step 4: Normalization and Data Preparation

# Normalize node features (standard normalization)
node_features = (node_features - node_features.mean(dim=0)) / node_features.std(dim=0)

# Normalize edge attributes (min-max normalization)
edge_attr = (edge_attr - edge_attr.min()) / (edge_attr.max() - edge_attr.min())

# Prepare the data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=labels)


In [19]:
# Step 5: Train the Model

model = SuperKAGCN(num_node_features=node_features.shape[1], hidden_dim=64, output_dim=1, dropout=0.3)
model = train_model(model, data, labels, num_epochs=500, patience=20)

Epoch 0, Loss: nan
Epoch 10, Loss: nan
Early stopping at epoch 19


TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.

In [42]:
# Step 6: Make Predictions

model.eval()
with torch.no_grad():
    predictions = model(data.x, data.edge_index, data.edge_attr)

# Interpret the predictions
for i, pred in enumerate(predictions):
    country = countries[i]
    change = 'Increase' if pred.item() > 0 else 'Decrease'
    print(f'Country: {country}, Predicted Change in Exchange Rate: {change}, Value: {pred.item()}')

Country: USA, Predicted Change in Exchange Rate: Decrease, Value: -0.0176653154194355
Country: Italy, Predicted Change in Exchange Rate: Increase, Value: 0.0027642855420708656
Country: UK, Predicted Change in Exchange Rate: Increase, Value: 0.022893346846103668
Country: Germany, Predicted Change in Exchange Rate: Decrease, Value: -0.011166456155478954
Country: Canada, Predicted Change in Exchange Rate: Increase, Value: 0.0040656146593391895
Country: France, Predicted Change in Exchange Rate: Decrease, Value: -0.004892876371741295
